## Assemble Training Dataset

This notebook unzips the needed files from the downloaded data from University of Mannheim - Research Group Data and Web Science - RealWorld (HAR) data:
https://sensor.informatik.uni-mannheim.de/#dataset_realworld

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import zipfile

In [ ]:
# unzip accelerometer files
# several files had "_2" inserted; one was missing

activities =["climbingdown","climbingup","jumping","lying","running","sitting","standing","walking"]
for subject in range(1,11):  # extract subjects
    zipfilepath="./data/proband"+str(subject)+"/data/"
    for activity in activities:
        zipfilename = "acc_"+activity+"_csv.zip"
        if (subject==4 and activity=='walking'):
            filename ="acc_walking_2_thigh.csv" 
        elif (subject in [6,7] and activity=='sitting'):
            filename ="acc_sitting_2_thigh.csv"
        elif (subject ==8 and activity=='standing'):
             filename ="acc_standing_2_thigh.csv"
        elif (subject==6 and activity=='jumping'):  #thigh data is missing, but other locations are available
            continue # skip this iteration and move to the next
        else:
            filename = "acc_"+activity+"_thigh.csv"
        savepath = "./data/"+str(subject)+"/"
        with zipfile.ZipFile(zipfilepath+zipfilename,"r") as zip_ref:
            zip_ref.extract(filename, path=savepath)

In [62]:
# unzip accelerometer files
# several files had "_2" inserted; one was missing

sensor = "gyr_"
activities =["climbingdown","climbingup","jumping","lying","running","sitting","standing","walking"]
for subject in range(1,11):  # extract subjects
    zipfilepath="./data/proband"+str(subject)+"/data/"
    for activity in activities:
        zipfilename = sensor+activity+"_csv.zip"
        if subject==1 and activity=="sitting":
            continue
        elif subject==4 and activity=="walking":
            filename = "Gyroscope_walking_2_thigh.csv"
        elif subject==6 and activity=="jumping":
            continue
        elif (subject in [6,7] and activity=='sitting'):
            filename ="Gyroscope_sitting_2_thigh.csv"
        elif (subject ==8 and activity=='standing'):
             filename ="Gyroscope_standing_2_thigh.csv"
        else:
            filename = "Gyroscope_"+activity+"_thigh.csv"
        savepath = "./data/"+str(subject)+"/"
        with zipfile.ZipFile(zipfilepath+zipfilename,"r") as zip_ref:
            zip_ref.extract(filename, path=savepath)

In [19]:
# rename _2 files
import os
os.rename("./data/4/acc_walking_2_thigh.csv", "./data/4/acc_walking_thigh.csv")
os.rename("./data/6/acc_sitting_2_thigh.csv", "./data/6/acc_sitting_thigh.csv")
os.rename("./data/7/acc_sitting_2_thigh.csv", "./data/7/acc_sitting_thigh.csv")
os.rename("./data/8/acc_standing_2_thigh.csv", "./data/8/acc_standing_thigh.csv")
os.rename("./data/4/Gyroscope_walking_2_thigh.csv", "./data/4/Gyroscope_walking_thigh.csv")
os.rename("./data/6/Gyroscope_sitting_2_thigh.csv", "./data/6/Gyroscope_sitting_thigh.csv")
os.rename("./data/7/Gyroscope_sitting_2_thigh.csv", "./data/7/Gyroscope_sitting_thigh.csv")
os.rename("./data/8/Gyroscope_standing_2_thigh.csv", "./data/8/Gyroscope_standing_thigh.csv")

In [80]:
# merge all activities into 1 df, for subjects 1-10, with gyroscope & accelerometer sensors combined
thigh = pd.DataFrame()  # initialize empty df
windowsize = 100 # use 2 sec windows
last_sample = 0
activities =["climbingdown","climbingup","jumping","lying","running","sitting","standing","walking"]
# iterate over subjects
for subject in range(1,11):
    # iterate over the separate activity files
    for i, activity in enumerate(activities):
        if subject==6 and activity=="jumping":
            continue # missing data for subject 6 acc data from thigh (other locations available)
        g_filename = "./data/"+str(subject)+"/Gyroscope_"+activity+"_thigh.csv"
        a_filename = "./data/"+str(subject)+"/acc_"+activity+"_thigh.csv"
        gyr = pd.read_csv(g_filename, index_col=0)
        acc = pd.read_csv(a_filename, index_col=0)
        # make sure the different sensor files have the same number of readings
        if len(gyr)>len(acc):
            extra_records=len(gyr)-len(acc)
            acc = acc[:-extra_records]
        if len(gyr)<len(acc):
            extra_records=len(acc)-len(gyr)
            gyr = gyr[:-extra_records]
        df = pd.merge(gyr,acc,right_index=True,left_index=True,suffixes=('_gyr','_acc'))
        df['activity']=activity
        df['label']=i
        df['subject']=subject
        df['sample_num']=-1  # just to create the column   
        remainder=len(df)%windowsize
        df=df[:-remainder]  # delete rows so that all samples are the same size
        num_samples=int(len(df)/windowsize)  # the number of samples of size windowsize for the activity
        row_counter=0
    # label the samples within each activity file    
        for j in range(0,num_samples):   
            df.sample_num[row_counter:row_counter+windowsize]=last_sample+j # label samples
            row_counter=row_counter+windowsize
        last_sample+=num_samples # label the samples consecutively across activities
        thigh = thigh.append(df, ignore_index=True)

C:\Users\pauls\Anaconda3\envs\dsi\lib\site-packages\ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [81]:
thigh.to_pickle('./thigh100.pkl')

In [76]:
thigh.subject.value_counts()

5     246150
8     244650
3     228300
9     225150
6     211800
10    210750
2     208050
4     205500
7     191400
1     188250
Name: subject, dtype: int64

In [49]:
# count number of records by subject, activity, and sensor
record_counts = []  # initialize empty list
activities =["climbingdown","climbingup","jumping","lying","running","sitting","standing","walking"]
# iterate over subjects
for subject in range(1,10):
    # iterate over the separate activity files
    for i, activity in enumerate(activities):
        if subject==1 and activity=="sitting":
            continue # missing data for this subject / activity
        if subject==6 and activity=="jumping":
            continue # missing data for this subject / activity
        g_filename = "./data/"+str(subject)+"/Gyroscope_"+activity+"_thigh.csv"
        a_filename = "./data/"+str(subject)+"/acc_"+activity+"_thigh.csv"
        with open(g_filename) as f:
            gyr_count = sum(1 for line in f)
        with open(a_filename) as f:
            acc_count = sum(1 for line in f)
        record_counts.append({'subject':subject,
                              'activity':activity,
                              'gyr_count':gyr_count,
                              'acc_count':acc_count})
records=pd.DataFrame(record_counts)